# TP Relation Extraction

## Introduction

Dans ce TP nous allons:

1. Découvrir la tache d’extraction de relation par réseaux neuronaux  (« neural relation extraction », NRE)
2.  Comprendre le fonctionnement d’un modèle d’extraction de relation avec un encodeur BERT
3. Découvrir la tache de reconnaissance d'entités nommées (« named-entity recognition », NER)
4. Coder une pipeline d’extraction de relation par réseaux neuronaux pour des jeux de données textuels

Avec les outils suivants:
1. La libraire [OpenNRE](https://github.com/thunlp/OpenNRE), basée sur Pytorch et HuggingFace’s Transformers, pour la tache d’extraction de relation par réseaux neuronaux
2. [HuggingFace’s Transformers](https://huggingface.co/transformers/) : une bibliothèque basée sur Pytorch pour le traitement automatique des langues et notamment les modèles neuronaux de type Transformer (comme BERT)
3. Google Colab, qui héberge ce *Jupyter Notebook*. Avant de commencer le TP, vous pouvez consulter des pages d'introductions [à Colab](https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l01c01_introduction_to_colab_and_python.ipynb#scrollTo=YHI3vyhv5p85) et [aux Notebooks](https://realpython.com/jupyter-notebook-introduction/)


Contrairement au dernier TP, cette fois-ci nous n’aurons pas besoin d’utiliser une GPU car nous n’allons pas entraîner des nouveaux modèles ou faire de l’inférence sur des grands jeux de données: une CPU suffira. Nous pouvons vérifier si l’on est en train d’utiliser une CPU ou une GPU avec les lignes suivantes:

In [1]:
import torch

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()
else:
  print("Will work on CPU.")

Will work on CPU.


Nous avons besoin d’installer l’outil OpenNRE. Pour éviter de devoir ré-télécharger le répertoire  GitHub de OpenNRE à chaque fois qu’on ré-initialise le fichier Colab, il est convenant de monter notre répertoire Google Drive et y télécharger le répertoire OpenNRE de façon à l’avoir toujours disponible. Ainsi, à chaque fois que nous allons ré-initialiser le fichier Colab, il nous suffira de monter notre Goodle Drive pour avoir accès à la libraire OpenNRE.

> ATTENTION: modifiez *tp_path_in_drive* pour pointer sur le repertoire où vous avez placé le fichier tp_re.ipynb, vous allez télécharger OpenNRE dans le meme réépertoire. Si vous êtes sur votre machine locale, vous n'avez pas besoin de monter le Drive, mais juste de faire le clonage du réépertoire OpenNRE.

In [ ]:
%%shell
cd drive/MyDrive/tp_re
ls

tp_re.ipynb


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
from google.colab import drive

tp_path_in_drive = '/drive/MyDrive/tp_re'
opennre_path_in_drive = tp_path_in_drive + '/OpenNRE'

# mount Google Drive
drive.mount('/content/drive')

if not os.path.isdir(opennre_path_in_drive):
  # OpenNRE is not already present in Google Drive
  if not os.path.isdir(tp_path_in_drive):
    # make directory for the TP if necessary
    os.makedirs(tp_path_in_drive, exist_ok=True)
  # change directory to the TP directory
  os.chdir(tp_path_in_drive)
  # clone OpenNRE repo
  print("Cloning repo...")
  os.system('git clone https://github.com/thunlp/OpenNRE.git')
  print("...done!")
else:
  print("OpenNRE is already present in Google Drive under {0}".format(opennre_path_in_drive))

# Change current dir to OpenNRE
os.chdir(opennre_path_in_drive)

Mounted at /content/drive
Cloning repo...
...done!


In [3]:
# Update requirements requirements
!sed -i '/transformers==3.0.2/c\transformers==3.4.0' requirements.txt

Nous pouvons désormais continuer avec l’installation.

In [4]:
!pip install -r requirements.txt
!python setup.py install

     |████████████████████████████████| 748.8 MB 18 kB/s 
     |████████████████████████████████| 1.3 MB 56.0 MB/s 
     |████████████████████████████████| 234 kB 49.6 MB/s 
     |████████████████████████████████| 7.0 MB 50.3 MB/s 
     |████████████████████████████████| 1.5 MB 44.0 MB/s 
     |████████████████████████████████| 2.9 MB 25.2 MB/s 
     |████████████████████████████████| 895 kB 64.5 MB/s 
     |████████████████████████████████| 1.2 MB 53.6 MB/s 
     |████████████████████████████████| 749 kB 66.3 MB/s 
  Attempting uninstall: regex
    Found existing installation: regex 2019.12.20
    Uninstalling regex-2019.12.20:
      Successfully uninstalled regex-2019.12.20
  Attempting uninstall: pluggy
    Found existing installation: pluggy 0.7.1
    Uninstalling pluggy-0.7.1:
      Successfully uninstalled pluggy-0.7.1
  Attempting uninstall: torch
    Found existing installation: torch 1.10.0+cu111
    Uninstalling torch-1.10.0+cu111:
      Successfully uninstalled torch-1.10.0+

running install
running bdist_egg
running egg_info
creating opennre.egg-info
writing opennre.egg-info/PKG-INFO
writing dependency_links to opennre.egg-info/dependency_links.txt
writing top-level names to opennre.egg-info/top_level.txt
writing manifest file 'opennre.egg-info/SOURCES.txt'
adding license file 'LICENSE'
writing manifest file 'opennre.egg-info/SOURCES.txt'
installing library code to build/bdist.linux-x86_64/egg
running install_lib
running build_py
creating build
creating build/lib
creating build/lib/opennre
copying opennre/pretrain.py -> build/lib/opennre
copying opennre/__init__.py -> build/lib/opennre
creating build/lib/opennre/tokenization
copying opennre/tokenization/utils.py -> build/lib/opennre/tokenization
copying opennre/tokenization/word_piece_tokenizer.py -> build/lib/opennre/tokenization
copying opennre/tokenization/bert_tokenizer.py -> build/lib/opennre/tokenization
copying opennre/tokenization/basic_tokenizer.py -> build/lib/opennre/tokenization
copying opennre

## Les modéles pour l'extraction de relation

À part offrir un cadre pour l’implémentation et l’entraînement de modelés pour l’extraction de relation, OpenNRE offre aussi des modèles déjà entraînés sur différents jeux de données, et donc capables de détecter différents types de relations entre entités. 

Ici, nous allons employer un modèle entraîné sur Wiki80 (dataset introduit par [le papier OpenNRE](https://www.aclweb.org/anthology/D19-3029/)), un jeu de données contenant des phrases collectées sur Wikipedia et Wikidata, ainsi que des rélations entre leurs entités. Si vous voulez en savoir plus sur Wiki80, vous pouvez le télécharger avec le script [download_wiki80.sh](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/benchmark/download_wiki80.sh) 

In [16]:
import opennre
model = opennre.get_model('wiki80_bert_softmax')


2022-03-20 09:05:32,458 - root - INFO - Loading BERT pre-trained checkpoint.


Nous pouvons utiliser ce modèle pour calculer la relation entre un mot «tête» et un mot «queue» qui sont contenus dans un texte. Il suffit de passer au modèle le texte ainsi que la position de la tête et de la queue. Le modèle retournera la relation de la tête à l’égard de la queue ainsi que la probabilité qu’il associe à cette rélation. Par example, nous pouvons inféérer la relation entre **Áed Uaridnach** et **Máel Dúin mac Máele Fithrich** de la façon suivante:

In [6]:
item = {'text': 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', 'h': {'pos': (78, 91)}, 't': {'pos': (18, 46)}}
print(model.infer(item))

('child', 0.9812852144241333)


In [7]:
txt = "sdfkjssdgfqk qksgfiuedje kjyzeyvd zgvd klzgidèe bzfsccskzez klqcshhgbsdjk jzdu kzjgd ozhd."
txt[txt.index('kzjgd'): txt.index('kzjgd')+ len('kzjgd')]
txt0 = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'

#### Exercice 1



Écrire une fonction `to_input_format(text, head, tail)` qui nous permet de trouver la position de deux mots (une tête et une queue) dans un texte, et qui retourne un dictionnaire contenant le texte et les deux positions suivant le format requis par la fonction `model.infer()`.

In [8]:
import re

def to_input_format(text, head, tail):
  """
  Args:
    text: a string of text
    head: a string of text representing a word contained in text
    tail: a string of text representing a word contained in text but different from head
  Returns:
    A dictionary containing the text and the position of head and tail within it,
    following the input format required by a SoftmaxNN model.
  """
  dic = {}
  dic['text'] = text
  dic['h'] = {'pos': (text.index(head), text.index(head) + len(head))}
  dic['t'] = {'pos': (text.index(tail), text.index(tail) + len(tail))}
  return dic
  # ....

In [9]:
# Test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
head = 'Áed Uaridnach'
tail = 'Máel Dúin mac Máele Fithrich'
test_item = to_input_format(text, head, tail)
try:
  assert model.infer(item) == model.infer(test_item)
  print("Good job!")
except AssertionError:
  print("Something is wrong with your function, try again!")

Good job!


#### Exercice 2

Décrire l’architecture du modèle qu’on vient de télécharger, la logique et le fonctionnement de chacun de ses composants, en faisant de références à ce que vous avez vu dans le cours ainsi que ce que vous avez appris dans le TP BERT.

Aide: vous pouvez inspecter un modèle Pytorch avec `print(model)`. Pour mieux le comprendre, vous pouvez voir aussi son [code](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/model/softmax_nn.py#L5).

La question a été mise à jour entre-temps pour être plus détaillée. Voir réponses ci-dessous.

#### Exercice 2

Décrire l’architecture du modèle qu’on vient de télécharger, la logique et le fonctionnement de chacun de ses composants:

1. Sentence-encoder: pourquoi l'on utilise un encodeur type BERT ?
2. À quoi sert la couche BertPooler ?
3. À quoi sert la couche linéaire finale (fc) ? pourquoi elle réduit à 80 la dimension du vecteur sortant de l’encodeur?
4. À quoi sert la fonction Softmax ?
5. À quoi sert la fonction de dropout que l'on appliuque à la sortie du réseau, ainsi que dans chaque couche de BERT ?

Aide: vous pouvez inspecter un modèle Pytorch avec print(model). Pour mieux le comprendre, vous pouvez aussi voir [son code](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/model/softmax_nn.py#L5), [la déscription du modèle BERT de Huggingface](https://huggingface.co/docs/transformers/v4.16.2/en/model_doc/bert#transformers.BertModel) ainsi que [son code](https://github.com/huggingface/transformers/blob/v4.16.2/src/transformers/models/bert/modeling_bert.py#L848).

In [70]:
print(model)

SoftmaxNN(
  (sentence_encoder): BERTEncoder(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
        

1) C'est un encodeur qui est adapté à notre cas, puisque un encodeur de type BERT est un encodeur pré-entrainé qui est sert à des modèles de multi-classification en avale. **Pourquoi est-il utile pour ce type de problème? Car il est capable de représenter la signification de chaque mot en fonction des mots qui l'entourent dans la même phrase (représentation contextualisée), c'est qui est très utile pour une comprehension générale de la phrase et sa classification. D'ailleurs, les représentations contextualisées sortant de BERT peuvent être utilisées pour beaucoup d'autres taches, et non-seulement la multi-classification**

2) La couche BertPooler est la dernière couche des hidden layers. Elle prend la représentation de l'output du premier token et l'utilise dans les tâches suivantes. 

3) Cette couche est la dernière couche du réseau. Elle sert à redimensionner l'output de telle façon à ce qu'il devient utilisable par la suite, il le réduit à la taille du nombre de classe qu'on souhaite trouver.

4) SoftMax sert à générer la probabilité d'exactitude des tokens prédit. **Le modèle ne prédit pas des tokens. La distribution de probas générée se référe aux 80 rélations (classes) possible pour la couple head-tail.**

5) Dropout sert à bloquer l'ouput de certains neurones aléatoirement, dans le but d'obliger le réseau, dans son processus d'apprentissage, à uniformiser les mises à jour des paramètres sur tous les neurones et éviter le sur-apprentissage.

**ce n'est pas claire comment le dropout empêche le sur-apprentissage** 

## Encodeur

#### Exercice 3

Maintenant, nous allons essayer de mieux comprendre le fonctionnement de ce modèle avec un focus sur son encodeur, qui est définit dans le fichier [bert_encoder.py](https://github.com/thunlp/OpenNRE/blob/60a8ceb42e1cfacbde3c8cfb5f758fb7fe96bdc4/opennre/encoder/bert_encoder.py#L7). Ce qui est spécifique à la tache de NRE dans ce modèle n’est pas l’architecture, mais plutôt la façon dont la séquence en input est tokenisée. Avec l’objectif de bien comprendre comment cet encodeur gère son input, répondez en détail aux questions suivantes qui se référent à la méthode `tokenize`. Si vous utilisez du code pour vous aider à répondre (conseillé), veuillez le joindre à vos réponses textuelles.

1. Qu’est-ce qu’elle sont les variables `sent0`, `ent0`, `sent1`, `ent1`, `sent2` ?
2. Qu’est-ce que c’est `re_tokens` ?
3. Dans le _forward_, le `BERTEncoder` prend en entrée exclusivement la séquence textuelle qui a étée préalablement tokenisée. Comment est-il capable de distinguer la tête et la queue en sort de (apprendre à) prédire la relation entre les deux ?
4. Qu’est-ce que c’est le « Padding » et pourquoi est-il utile ?
5. Qu’est-ce que c’est l’ « Attention mask » et pourquoi est-elle utile ?
6. **[BONUS]** Quelle est la différence en termes de fonctionnement entre la classe `BERTEncoder` et la classe `BERTEntityEncoder`, définie dans le même fichier ?


Aide : vous pouvez accéder à la méthode `tokenize` pour la tester avec `model.sentence_encoder.tokenize(item)`

In [ ]:
# Write your code here if needed, and then write your detailed answers below.
print(model.sentence_encoder.tokenize(item))

1) Si on considère que **first** est celui, entre *head* et *tail*, qui se positione le premier dans la phrase et **last** celui restant.


* `sent0`: Est le bout de texte, de l'input, se situant avant first.

* `ent0` : C'est **first**.

* `sent1`: Le bout de texte entre **first** et **last**.

* `ent1` : C'est **last**.

* `sent2`: Le bout de texte si situant après **last**.

Dans toutes les variables ci-dessus, on stocke les bouts de texte sous forme de tockens.

2) `re_tokens` représente la version finale de notre liste de tokens avec **'CLS'** comme premier token et **'SEP'** comme dernier.

3) 

```
ent0 = ['[unused0]'] + ent0 + ['[unused1]'] if not rev else ['[unused2]'] + ent0 + ['[unused3]']
ent1 = ['[unused2]'] + ent1 + ['[unused3]'] if not rev else ['[unused0]'] + ent1 + ['[unused1]']
```
Le bout de code ci-dessus est celui qui permet au *forward* de `BERTEncoder` de distinguer le *head* et le *tail* en les positionnant entre des tokens prédéfinis.

4) L'opération de padding est une opération de redimensionnement du vecteur des *tokens indéxés* en ajoutant des 0s jusqu'à attaindre le `max_length`. Cette opération a pour but d'uniformiser la taille des *tensors*, si c'est pas le cas on aura des erreurs à l'entrée du modèle. **Pourquoi est-il utile d'uniformer la taille? -> pour qu'on puisse representer un lot de phrases avec une matrice, ce qui est nécessaire au réseau pour traiter les phrases en parallèle.**
5) « Attention mask » est une liste de 0s et de 1s; Soit idx un indice, att_mask[idx] doit être égale à 1 si token[idx] doit être considéré/traité par la suite, et 0 sinon. Car après une opération de padding on aura dans la liste `token` des 0s qui sont que pour la forme, et qui ne doivent pas être inclus dans le calcul. Cela a pour effet de diminuer le temps de calcul.

6) 

## NRE Pipeline

Nous allons maintenant programmer une application qui prend en entrée une phrase et donne en sortie deux entités nommées dans la phrase ainsi que la relation entre eux.

Pour ce faire, nous avons besoin :

1. d’un système NER, qui reconnaît les entités nommées dans la phrase
2. d’un système NRE, comme celui que nous avons utilisé jusqu’à là

#### Exercice 4

En vous aident avec la documentation de HuggingFace, instanciez une [pipeline](https://huggingface.co/transformers/main_classes/pipelines.html?highlight=pipeline#the-pipeline-abstraction) `ner_pipeline` pour la reconnaissance d'entités nommées, avec le modèle et le tokeniseur pré-entraînes [dbmdz/bert-large-cased-finetuned-conll03-english](https://huggingface.co/dbmdz/bert-large-cased-finetuned-conll03-english).

Vous pouvez tester la bonne réussite de l’exercice avec le code ci-dessous.

**Indice** : la solution consiste en deux lignes de code : l'une pour importer la classe pipeline, l'autre pour instancier la bonne pipeline.

In [11]:
# Write your code here
from transformers import pipeline,AutoModelForTokenClassification, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
#tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")
model = AutoModelForTokenClassification.from_pretrained("dbmdz/bert-large-cased-finetuned-conll03-english")
ner_pipeline = pipeline("ner", model = model, tokenizer = tokenizer)

Downloading:   0%|          | 0.00/998 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.33G [00:00<?, ?B/s]

In [12]:
txt = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
ner_pipeline(txt)

[{'entity': 'I-PER', 'index': 6, 'score': 0.982085645198822, 'word': 'M'},
 {'entity': 'I-PER', 'index': 7, 'score': 0.8588601350784302, 'word': '##á'},
 {'entity': 'I-PER', 'index': 8, 'score': 0.8766007423400879, 'word': '##el'},
 {'entity': 'I-PER', 'index': 9, 'score': 0.8375428915023804, 'word': 'D'},
 {'entity': 'I-PER', 'index': 10, 'score': 0.4897182583808899, 'word': '##ú'},
 {'entity': 'I-PER', 'index': 11, 'score': 0.901253342628479, 'word': '##in'},
 {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
 {'entity': 'I-PER', 'index': 13, 'score': 0.8884768486022949, 'word': 'M'},
 {'entity': 'I-PER', 'index': 14, 'score': 0.7004077434539795, 'word': '##á'},
 {'entity': 'I-PER',
  'index': 15,
  'score': 0.8791088461875916,
  'word': '##ele'},
 {'entity': 'I-PER', 'index': 16, 'score': 0.9720047116279602, 'word': 'Fi'},
 {'entity': 'I-PER', 'index': 17, 'score': 0.697806715965271, 'word': '##th'},
 {'entity': 'I-PER',
  'index': 18,
  'score': 0.697088

In [13]:
# Apply your ner_pipeline to some sentences to see how it works,
# Then you can test your code with this snippet
text = 'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).'
try:
  assert ner_pipeline(text) == [
                                {'entity': 'I-PER', 'index': 6, 'score': 0.982085645198822, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 7, 'score': 0.8588606715202332, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 8, 'score': 0.876600980758667, 'word': '##el'},
                                {'entity': 'I-PER', 'index': 9, 'score': 0.8375431299209595, 'word': 'D'},
                                {'entity': 'I-PER', 'index': 10, 'score': 0.4897184371948242, 'word': '##ú'},
                                {'entity': 'I-PER', 'index': 11, 'score': 0.9012535810470581, 'word': '##in'},
                                {'entity': 'I-PER', 'index': 12, 'score': 0.5821399688720703, 'word': 'mac'},
                                {'entity': 'I-PER', 'index': 13, 'score': 0.8884767889976501, 'word': 'M'},
                                {'entity': 'I-PER', 'index': 14, 'score': 0.7004088759422302, 'word': '##á'},
                                {'entity': 'I-PER', 'index': 15, 'score': 0.8791091442108154, 'word': '##ele'},
                                {'entity': 'I-PER', 'index': 16, 'score': 0.9720048308372498, 'word': 'Fi'},
                                {'entity': 'I-PER', 'index': 17, 'score': 0.6978078484535217, 'word': '##th'},
                                {'entity': 'I-PER', 'index': 18, 'score': 0.6970881223678589, 'word': '##rich'},
                                {'entity': 'I-PER', 'index': 26, 'score': 0.9203091859817505, 'word': 'Á'},
                                {'entity': 'I-PER', 'index': 27, 'score': 0.9416706562042236, 'word': '##ed'},
                                {'entity': 'I-PER', 'index': 28, 'score': 0.9702795147895813, 'word': 'U'},
                                {'entity': 'I-PER', 'index': 29, 'score': 0.8428964614868164, 'word': '##ari'},
                                {'entity': 'I-PER', 'index': 31, 'score': 0.707791805267334, 'word': '##ach'}
                                ]
  print("Good job!")
except AssertionError:
  print("Something might be wrong with your pipeline.") 

Something might be wrong with your pipeline.


#### Exercice 5


Nous pouvons finalement développer une pipeline NRE reposante sur notre modèle de NRE et notre pipeline NER. 

Écrivez ci-dessous une classe `NREPipeline` équipée (entre autres) d'une méthode `__call__(self, text)` qui prend un texte en entrée et effectue les opérations suivantes :

- elle reconnaît les entités dans le texte
    - retenir seulement les deux entités auxquelles la pipeline NER associe la probabilité la plus élevée, écarter les autres (si presentes)
    - si la pipeline NER ne reconnaît qu’une seule entités dans le texte, `__call__(self, text)` retourne None (voir le test en bas) car il n’y a aucune relation à prédire
- elle donne en sortie une liste en format `[e1, e2, rel, p]` où :
    - `e1` est la première entité reconnue dans le texte (entre les deux plus probables, la première qui apparaître dans le texte)
    - `e2` est la deuxième entité  reconnue dans le texte (entre les deux plus probables, la deuxième qui apparaître dans le texte)
    - `rel` est la relation qu’il y a entre `e1` et `e2`
    - `p` est la probabilité associée à la relation `rel` par le modèle de NRE

Pour vérifier la bonne qualité de votre classe `NREPipeline`, utilisez l’extrait de code ci-dessous. Le résultat devrait ressabler à celui-ci :

````
Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ['Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234]
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ['Iwami Ginzan', 'World Heritage Site', 'heritage designation', 0.9991846680641174]
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ['Earleville', "St . Stephen ' s Episcopal Church", 'location', 0.9127373099327087]
------------------------------
````

In [65]:
# Write your pipeline in this cell. The fun begins here, because we are coding a whole Python class here !
from itertools import combinations
class NREPipeline(object):
  def __init__(self, ner_pipeline, nre_model):
    # ...
    self.ner = ner_pipeline
    self.nre = nre_model

  # ...

  def __call__(self, text):
    ner_output = ner_pipeline(text)
    # discard 'UNK'
    ner_output = [elem for elem in ner_output if elem['word'] != '[UNK]']
    index_word = [(i['index'], i['word']) for i in ner_output]
    # assemble entities from ner_output
    i = 1
    while i < len(index_word):
      if index_word[i -1][0] + 1 != index_word[i][0] or index_word[i][1][:2] != "##":
        index_word.insert(i, (index_word[i -1][0] + 1, ' '))
        i += 1
      i += 1
    entities = []
    start = 0
    for i in range(1, len(index_word)):
      if index_word[i - 1][1] == ' ' and index_word[i - 1][0] != index_word[i][0]:
        entities.append([index_word[j][1] for j in range(start, i - 1)])
        start = i
    
    entities.append([index_word[j][1] for j in range(start, len(index_word))])
    entities = [''.join(l).replace('##', '') for l in entities]
    entities = [''.join(l).replace(" ' ", " '") for l in entities]
    entities = [''.join(l).replace(" . ", ". ") for l in entities]
    res = (0, 0)
    if len(entities) > 1:
      for elem in combinations(entities, 2):
        # print(elem[0], '|', elem[1])
        item = to_input_format(text, elem[0], elem[1])
        if model.infer(item)[1] > res[1]:
          res = model.infer(item)
          elemres = elem
      if elem[0] == "Earleville" and elem[1] == "St. Stephen 's Episcopal Church":
        print(model.infer(item))
      return list(elemres + res)
    else:
      return None
    
   # ...

In [33]:
# Test your code with this snippet

sequences = [
             'He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).', # Easy sentence
             'He was the son of Máel Dúin', # There is only one entity in this sentence, therefore our pipeline should return None
             'Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .', # Ōda is tokenized by the NER tokenizer as a "[UNK]" and it is detected as an entitity. For simplicity, you can discard [UNK] entities as if they were not detected.
             'It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .', # the NER system can not recognise any entity here, therefore the pipeline should return None
             "Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church ." # This sentence is not trivial because the 's has to be managed properly

]

nre = NREPipeline(ner_pipeline, model)
for n, sequence in enumerate(sequences):
  out = nre(sequence)
  print("Sentence {0}: {1}".format(n, sequence))
  print("System out: ", out)
  print('------------------------------')

Sentence 0: He was the son of Máel Dúin mac Máele Fithrich, and grandson of the high king Áed Uaridnach (died 612).
System out:  ('Máel Dúin mac Máele Fithrich', 'Áed Uari', 'father', 0.9923498034477234)
------------------------------
Sentence 1: He was the son of Máel Dúin
System out:  None
------------------------------
Sentence 2: Ōda is home to the Ōda Iwami Ginzan Silver Mine , a World Heritage Site .
System out:  ('Iwami Ginzan', 'World Heritage Site', 'heritage designation', 0.9992959499359131)
------------------------------
Sentence 3: It has been shown to be equally effective as leuprorelin , which is a second - line medication against endometriosis .
System out:  None
------------------------------
Sentence 4: Located at Earleville and listed on the National Register of Historic Places are : Bohemia Farm , Mount Harmon , Rose Hill , and St. Stephen 's Episcopal Church .
System out:  ('Earleville', 'National Register of Historic Places', 'heritage designation', 0.99950230121

## Application de le pipeline

#### Exercice 6

1. Appliquez la pipeline NRE aux phrases contenues dans le fichier _sentences.txt_, qui ont étées extraites à partir de Wikipedia et Wikidata.

2. Donnez un avis qualitatif sur sa performance, les problèmes rencontrés ainsi que des idées pour améliorations des résultats (e.g. entraînement sur des données différentes, modèle différent, etc.).

3. Appliquez la pipeline NRE aux premières 50 phrases contenues dans le fichier _Sentences_AllAgree.txt_ que vous avez utilisé pour  TP BERT, qui parlent d’événements financiers

4. Remarquez-vous des différences en terme de performances par rapport à la question 2 ? Pourquoi ? Commentez…

In [ ]:
# Write your code here and then write your detailed answers below. Do not hesitate to use more code cells if needed

In [39]:
txt = "Along the I-5 stretch , Walter Wirth Lake and McNary Field ( Salem Municipal Airport ) are on the left ; near Route 22 , the unincorporated neighborhood of Four Corners is on the right ."
ner_output = ner_pipeline(txt)
ner_output = [elem for elem in ner_output if elem['word'] != '[UNK]']
lm = [(i['index'], i['word']) for i in ner_output]
i = 1
while i < len(lm):
  # print(lm[i - 1][0], lm[i][0])
  if lm[i -1][0] + 1 != lm[i][0] or lm[i][1][:2] != "##":
    lm.insert(i, (lm[i -1][0] + 1, ' '))
    i += 1
  i += 1
lf = []
idx = 0
for i in range(1, len(lm)):
  if lm[i - 1][1] == ' ' and lm[i - 1][0] != lm[i][0]:
    lf.append([lm[j][1] for j in range(idx, i - 1)])
    idx = i
lf.append([lm[j][1] for j in range(idx, len(lm))])
lf = [''.join(l).replace('##', '') for l in lf]
lf = [''.join(l).replace(" ' ", " '") for l in lf]
lf = [''.join(l).replace(" . ", ". ") for l in lf]
print(ner_pipeline(txt))
print(lm)
print(lf)

[{'word': 'I', 'score': 0.6765465140342712, 'entity': 'I-MISC', 'index': 3}, {'word': '-', 'score': 0.5596017837524414, 'entity': 'I-MISC', 'index': 4}, {'word': '5', 'score': 0.7006650567054749, 'entity': 'I-MISC', 'index': 5}, {'word': 'Walter', 'score': 0.4088774025440216, 'entity': 'I-PER', 'index': 8}, {'word': 'W', 'score': 0.6508683562278748, 'entity': 'I-LOC', 'index': 9}, {'word': '##irt', 'score': 0.23423130810260773, 'entity': 'I-ORG', 'index': 10}, {'word': '##h', 'score': 0.35906338691711426, 'entity': 'I-ORG', 'index': 11}, {'word': 'Lake', 'score': 0.8282280564308167, 'entity': 'I-LOC', 'index': 12}, {'word': 'M', 'score': 0.8583234548568726, 'entity': 'I-LOC', 'index': 14}, {'word': '##c', 'score': 0.7494085431098938, 'entity': 'I-LOC', 'index': 15}, {'word': '##N', 'score': 0.4825979471206665, 'entity': 'I-LOC', 'index': 16}, {'word': '##ary', 'score': 0.7962921261787415, 'entity': 'I-LOC', 'index': 17}, {'word': 'Field', 'score': 0.9498450756072998, 'entity': 'I-LOC',

In [69]:
fname = '../../../../content/sentences.txt'
f = open(fname, 'rt', encoding="ISO-8859-1")
n = 1
nb_sambles = 46
mean_proba = 0
nre = NREPipeline(ner_pipeline, model)
for line in f:
  # On élimine 4 phrases qui créent des problèmes, et ne passent pas dans l'algo
  if n not in (7, 29, 37, 42):
    out = nre(line)
    print("Sentence {0}: {1}".format(n, line))
    print("System out: ", out)
    print('------------------------------')
    if out != None:
      mean_proba += out[3]
    else:
      nb_sambles -= 1
  n += 1
mean_proba /= nb_sambles
print('la précision moyenne est :', mean_proba)

Sentence 1: The Willard and Josephine Hubbard House was individually listed on the National Register of Historic Places in 2016 .

System out:  ['Josephine Hubbard House', 'National Register of Historic Places', 'heritage designation', 0.9995538592338562]
------------------------------
Sentence 2: His station commander , Group Captain Claude Hilton Keith , found a letter among the missing airman 's personal possessions .

System out:  None
------------------------------
Sentence 3: One was Quintus Caecilius Metellus Creticus , who was praetor in 74 BC and consul in 69 BC .

System out:  None
------------------------------
Sentence 4: April 2009 In addition to musical acts , the label recorded beat poets Lawrence Ferlinghetti and Allen Ginsberg and comic Lenny Bruce .

System out:  ['Lawrence Ferlinghetti', 'Lenny Bruce', 'sibling', 0.828484058380127]
------------------------------
Sentence 5: Walter Neusel ( November 25 , 1907 â October 3 , 1964 ) was a German heavyweight boxer .

Sy

In [71]:
fname = '../../../../content/Sentences_AllAgree.txt'
f = open(fname, 'rt', encoding="ISO-8859-1")
n = 1
nre = NREPipeline(ner_pipeline, model)
nb_sambles = 127
mean_proba = 0
for line in f:
  if n not in (20, 38, 47, 50, 81, 85, 122):
    out = nre(line)
    print("Sentence {0}: {1}".format(n, line))
    print("System out: ", out)
    print('------------------------------')
    if out != None:
      mean_proba += out[3]
    else:
      nb_sambles
  n += 1
mean_proba /= nb_sambles
print('la précision moyenne est :', mean_proba)

Sentence 1: According to Gran , the company has no plans to move all production to Russia , although that is where the company is growing .@neutral

System out:  ['Gran', 'Russia', 'owned by', 0.881830096244812]
------------------------------
Sentence 2: For the last quarter of 2010 , Componenta 's net sales doubled to EUR131m from EUR76m for the same period a year earlier , while it moved to a zero pre-tax profit from a pre-tax loss of EUR7m .@positive

System out:  ['EU', 'EU', 'applies to jurisdiction', 0.8518947958946228]
------------------------------
Sentence 3: In the third quarter of 2010 , net sales increased by 5.2 % to EUR 205.5 mn , and operating profit by 34.9 % to EUR 23.5 mn .@positive

System out:  ['EU', 'EU', 'member of', 0.6252642869949341]
------------------------------
Sentence 4: Operating profit rose to EUR 13.1 mn from EUR 8.7 mn in the corresponding period in 2007 representing 7.7 % of net sales .@positive

System out:  ['EU', 'EU', 'applies to jurisdiction', 0

2) Pour le premier DataSet on trouve que la précision moyenne est de 0.9703475070554156, ce qui est très bon.
Le soucis qu'on trouve, comme indiqué dans un commentaire du code, est que quelques phrases du DataSet n'arrivent pas à être traité par notre Algo.
Par exemple, pour la phrase n° 7, "I-5" n'est pas considéré comme un nom en soit par la pipeline NER ce qui crée des problèmes par la suite.

Comme réctification, on propose de trouver un tokenizer plus précis (par soucis de temps cela ne sera pas fait).

3) Rmq: on n'utilise qu'une partie du DataSet à savoir 134 phrases, tout en éliminant les phrases qui bloquent l'Algo. Cela pour avoir un résultat sur la précision comparable à celui trouvé pour le premier.

4) Pour le deuxième DataSet on trouve que la précision moyenne est de 0.5208455392225521, ce qui est nettement moins bien que le premier résultat.

Cela pourrait être dû au fait que les phrases sont ciblés sur un domaine bien précis à savoir la finance. Donc on retrouve des mots qui sont bien spécifiques au domaine, mais qui ne sont pas captés par la pipeline NER. On prend l'exemple de la devise "EUR" qui à chaque apparition n'est pas detectée mais c'est le sous mot "EU" qui l'est et qui référe à "European Union".

Donc une solution, serait d'entrainé le modèle NER sur des phrases inspirés du domaine financier pour pouvoir detecté ce genre de mots.


**Tout de même, le modèle d'extraction de relations devrait être entraîné avec des relations plus spécifiques, qui ne sont pas présentes dans l'ensemble des 80 relations définies par Wiki80.**